<a href="https://colab.research.google.com/github/jakubdaron/MTR_optymalizacja/blob/main/Problem_RPQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np

def generate_sorted_rpq_table(n, F):
    rpq = []
    # generate random r, p, q values
    for i in range(n):
        # always generate unique values for r since two tasks cannot occur at the same time
        while True:
            r = random.randint(0, n*F)
            if r not in [item[0] for item in rpq]:
                break

        p = random.randint(1, 50)
        q = random.randint(0, n*F)
        rpq.append([r, p, q])

    # sort rpq table in increasing order of r values
    rpq = np.array(rpq)
    sorted_rpq = rpq[rpq[:, 0].argsort()]
    #sorted_rpq = np.array([[6, 4, 2], [7, 8, 1], [10, 2, 3], [12, 3, 0]])
    return sorted_rpq

def add_and_sort_bufor(sorted_rpq, iteration, bufor_queue):
    bufor = sorted_rpq[iteration].tolist()
    bufor_queue.append(bufor)
    bufor_queue = np.array(bufor_queue)

    # sort bufor_queue in decreasing order of q values
    sorted_bufor = bufor_queue[bufor_queue[:, 2].argsort()[::-1]]
    bufor_queue = sorted_bufor.tolist()
    return bufor_queue

def optimise_function(sorted_rpq, n):
    iteration = 0
    bufor_queue = []
    current_task = 0
    time = 0
    end_iteration = 0
    end_def = 0

    while end_def == 0:
        #print(current_task)
        if time == sorted_rpq[iteration, 0]:
            if current_task <= 0 and len(bufor_queue) == 0:
                current_task = sorted_rpq[iteration, 1]
            elif current_task <= 0 and len(bufor_queue) > 0:
                bufor_queue = add_and_sort_bufor(sorted_rpq, iteration, bufor_queue)
                current_task = bufor_queue[0][1]
                bufor_queue.pop(0)
            else:
                bufor_queue = add_and_sort_bufor(sorted_rpq, iteration, bufor_queue)

            if iteration < n-1:
                iteration += 1
            else:
                end_iteration = 1

        elif current_task <= 0:
            if bufor_queue:
                current_task = bufor_queue[0][1]
                bufor_queue.pop(0)

        print(f"time: {time}, current_task: {current_task}, bufor_queue: {bufor_queue}")
        #print(f"time: {time}, current_task: {current_task}")
        time += 1
        current_task -= 1

        if end_iteration == 1:
            if current_task == 0 and len(bufor_queue) == 0:
                end_def = 1

if __name__ == "__main__":
    F = 17
    n = 20
    sorted_rpq = generate_sorted_rpq_table(n, F)
    print(sorted_rpq)
    optimise_function(sorted_rpq, n)